In [1]:
# 웹사이트
!pip install pyngrok -q
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.1 MB/s eta 0:00:00


In [2]:
# 스트림밋 시작
import streamlit as st
from pyngrok import ngrok

# https://ngrok.com/ 토큰 발행
my_token = '2X0EYF8JF2eQJbF76vX029bQM0F_2MTEtA1HELj7vnJ3NohxC'

# token 입력
ngrok.set_auth_token(my_token)
# ngork 초기화
ngrok.kill()

In [7]:
%%writefile app.py

import streamlit as st
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer


# 메인을 선택했나 안했나 체크
if "check" not in st.session_state:
    st.session_state.check = False

# 초기 줄거리 5개 리스트
if "main_story" not in st.session_state:
    st.session_state.main_story = ['군 복무를 마치고 복학한 만섭은 학교에 족구장이 사라졌다는 걸 알게 된다. 그는 이 문제를 해결하기 위해 대학교에 찾아가 족구장을 만들어 달라고 요구한다.',
'최초의 핵무기를 개발한 제2차 세계 대전 프로젝트인 맨해튼 프로젝트에서 역할로 "원자 폭탄의 아버지"로 인정받은 미국의 이론 물리학자 J. 로버트 오펜하이머',
'착하고 예쁜 신데렐라는 비록 귀족의 딸이나, 새엄마의 학대와 배다른 두 언니들의 심술로, 하녀와도 같은 혹독한 나날을 겪고 있다. 그러던 중 요정 대모의 도움으로 신데렐라도 왕궁의 무도회에 갈 수 있게 된다.',
'대한민국을 뒤흔든 10월 26일 이후, 서울에 새로운 바람이 불어온 것도 잠시 12월 12일, 보안사령관 전두광이 반란을 일으키고 군 내 사조직을 총동원하여 최전선의 전방부대까지 서울로 불러들인다.',
'토니 스타크는 뉴욕 사건을 겪은 뒤 슈퍼히어로의 삶에 회의감과 혼란스러움을 느낀다. 그런 그의 앞에 사악한 테러리스트 만다린이 나타나 스타크 저택을 파괴하고 토니의 주변인을 위협한다.'
]

# 초기 포스터 5개 url 리스트
if "main_story_image_url" not in st.session_state:
    st.session_state.main_story_image_url = ['https://i.namu.wiki/i/Bq2x4dxH-r6TlGZxbO-EPwz_Dk0zVfNo9o6n8YUNbzJLi6mBh0Bem4Q6Ww-A_7Srbf5mjSbqaj5LGeQxi-gEMA.webp',
'https://i.namu.wiki/i/rd5oNqqBY3tPw9hF3olu2hhI2Pyi5RwqpfhN_EUwDio-qhmHWa-Vu0VHD4iKk3g1iTbb_a_thcqkiiQSTLmlNQ.webp',
'https://i.namu.wiki/i/7W1qOWViBSDjavzWhlt_peSkrB9Lf2cQ7fEq92DQMLKu3L77347HN0r8zhkXYoriFcnnYoqRvDu97M1CJldW3Q.webp',
'https://i.namu.wiki/i/dQYtrYCT20iBXl8NVB32m-USGBNORan-hsEgwrGpEQeivNd-R-MTfLh0v01lmGV3CUc4PMCuW6BEswVt_Gt6MQ.webp',
'https://i.namu.wiki/i/RdPvwLtJeiOnKyXtjmqugaE6rxp_AlzJdxTi74oPxmr5Zi6w2gKuOHFiNWYayw3GQgukE_fX9CHM0s8FGKjrPQ.webp'
]

# 선택지 리스트
if "select_story" not in st.session_state:
    st.session_state.select_story = ['',
'',
'',
''
]

# 메인 스토리 쌓는 변수
if "main_story_input" not in st.session_state:
    st.session_state.main_story_input = ''

# 결과 변수(여기선 점수) 장르와 요약 변수로 변경해야함
if "result_story" not in st.session_state:
    st.session_state.result_story = ''

# 메인 리스트중 선택한 인덱스, 나중에 포스터 뽑으려고 생성함
if "main_select_index" not in st.session_state:
    st.session_state.main_select_index = 0

# 결과를 선택했는지, 결과를 선택하면 선택지 버튼 등 모두 안보이게 처리하려고 생성함
if "result_check" not in st.session_state:
    st.session_state.result_check = False

if "sub_title" not in st.session_state:
    st.session_state.sub_title = ['진행하고 하고자 하는 스토리를 선택하여 주세요', '선택지를 선택해주세요']


# 선택지 생성
def generate() -> str:
    st.session_state.select_story[0] = "1"
    st.session_state.select_story[1] = "2"
    st.session_state.select_story[2] = "3"
    st.session_state.select_story[3] = "결과"

# 웹페이지 시작
st.title('시나리오 제작 시뮬레이터')
st.subheader(st.session_state.sub_title[0])

# 포스터 컨테이너
col1, col2, col3, col4, col5 = st.columns(5)

# 포스터 클릭 함수
def main_stroy_click(index: int):
    with st.spinner('시나리오 작성중...'):
        st.session_state.main_select_index = index

        # 선택한 메인 스토리 인풋값에 넣기
        st.session_state.main_story_input = st.session_state.main_story[index]
        # 선택지 생성
        generate()
        st.session_state.check = True
        st.subheader(st.session_state.sub_title[1])
        st.rerun()

# 쌓인 메인 스토리 표현
st.write(st.session_state.main_story_input)

# 포스터 표현 버튼 없이 포스터를 선택하면 선택되도록 만들어야함..
if(st.session_state.check == False):
    with col1:
        st.image(st.session_state.main_story_image_url[0])
        if st.button('선택하기', key='button1', use_container_width=True):
            main_stroy_click(0)
    with col2:

        st.image(st.session_state.main_story_image_url[1])
        if st.button('선택하기', key='button2', use_container_width=True):
            main_stroy_click(1)

    with col3:
        st.image(st.session_state.main_story_image_url[2])
        if st.button('선택하기', key='button3', use_container_width=True):
            main_stroy_click(2)

    with col4:
        st.image(st.session_state.main_story_image_url[3])
        if st.button('선택하기', key='button4', use_container_width=True):
            main_stroy_click(3)

    with col5:
        st.image(st.session_state.main_story_image_url[4])
        if st.button('선택하기', key='button5', use_container_width=True):
            main_stroy_click(4)

# 선택지 클릭
def select_stroy_click(index: int):
    with st.spinner('시나리오 작성중...'):
        # 쌓인 메인 스토리에 띄어쓰기 더하기
        st.session_state.main_story_input += ' '

        # 쌓인 메인 스토리에 선택한 선택지 더하기
        st.session_state.main_story_input += st.session_state.select_story[index]
        generate()
        st.rerun()

# 결과 선택지 클릭
def result_click(index: int):
    with st.spinner('평점 작성중...'):
        st.session_state.main_story_input += ' '
        st.session_state.main_story_input += st.session_state.select_story[index]
        result_generate()
        st.rerun()

# 결과물 생성 함수 장르와 요약 변수로 변경해야함
def result_generate() -> str:
    # 결과 스토리 넣기
    st.session_state.result_story = "점수"


if(st.session_state.check and st.session_state.result_check==False):
    if st.button(st.session_state.select_story[0], use_container_width=True):
        select_stroy_click(0)
    if st.button(st.session_state.select_story[1], use_container_width=True):
        select_stroy_click(1)
    if st.button(st.session_state.select_story[2], use_container_width=True):
        select_stroy_click(2)
    if st.button('결론: ' + st.session_state.select_story[3], use_container_width=True):
        st.session_state.result_check = True
        result_click(3)

# 결과에 포스터 생성
if(st.session_state.result_check):
    st.image(st.session_state.main_story_image_url[st.session_state.main_select_index])

# 결과 표현
st.write(st.session_state.result_story)

Overwriting app.py


In [9]:
!nohup streamlit run app.py --server.port 80 &

nohup: appending output to 'nohup.out'


In [10]:
ngrok.connect(80)

<NgrokTunnel: "https://76ff-34-29-254-210.ngrok-free.app" -> "http://localhost:80">

In [8]:
# ngrok 초기화
ngrok.kill()

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.125.112.108
